# Model Registration for HP AI Studio

This notebook registers your trained model with MLflow for deployment in HP AI Studio.

## Instructions:
1. Update the configuration section with your model details
2. Run all cells in order
3. Verify model appears in HP AI Studio Deployments tab

In [ ]:
# Configuration - Update these values
MODEL_NAME = 'my-ai-model'
MODEL_VERSION = '1.0.0'
MODEL_PATH = './models/my_model'
MODEL_DESCRIPTION = 'Description of your AI model'
MLFLOW_TRACKING_URI = './mlruns'
EXPERIMENT_NAME = 'ai-560-student-projects'
STUDENT_NAME = 'Your Name'
PROJECT_TITLE = 'Your Project Title'

print(f'Configuration loaded for: {MODEL_NAME}')
print(f'Student: {STUDENT_NAME}')
print(f'Project: {PROJECT_TITLE}')

In [ ]:
import mlflow
import mlflow.pyfunc
from mlflow.models.signature import ModelSignature
from mlflow.types.schema import Schema, ColSpec
from mlflow.types import DataType
import pandas as pd
import torch
from datetime import datetime
import json
from pathlib import Path

print('Libraries imported successfully')

In [ ]:
class CustomModelWrapper(mlflow.pyfunc.PythonModel):
    """Wrapper class for MLflow model deployment"""
    
    def load_context(self, context):
        """Load model and dependencies"""
        # Add your model loading code here
        # Example: self.model = torch.load(context.artifacts['model_path'])
        print('Model loaded successfully')
    
    def predict(self, context, model_input):
        """Run inference"""
        # Add your prediction code here
        # Example: return self.model(model_input)
        return {'output': 'Model prediction would go here'}

print('Model wrapper class defined')

In [ ]:
# Define model signature
input_schema = Schema([ColSpec(DataType.string, 'input')])
output_schema = Schema([ColSpec(DataType.string, 'output')])
signature = ModelSignature(inputs=input_schema, outputs=output_schema)

# Create example input
input_example = pd.DataFrame({'input': ['example input data']})

print('Model signature defined')
print(f'Input schema: {input_schema}')
print(f'Output schema: {output_schema}')

In [ ]:
# Set MLflow tracking
mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)
mlflow.set_experiment(EXPERIMENT_NAME)

print(f'Registering model: {MODEL_NAME}')

# Start MLflow run
with mlflow.start_run(run_name=f"{MODEL_NAME}-{datetime.now().strftime('%Y%m%d-%H%M%S')}") as run:
    # Log parameters
    mlflow.log_param('model_version', MODEL_VERSION)
    mlflow.log_param('student_name', STUDENT_NAME)
    mlflow.log_param('project_title', PROJECT_TITLE)
    
    # Log model
    mlflow.pyfunc.log_model(
        artifact_path='model',
        python_model=CustomModelWrapper(),
        signature=signature,
        input_example=input_example,
        registered_model_name=MODEL_NAME
    )
    
    print(f'✓ Model registered: {MODEL_NAME}')
    print(f'✓ Run ID: {run.info.run_id}')
    print(f'✓ Check HP AI Studio Deployments tab')

In [ ]:
# Verify registration
client = mlflow.tracking.MlflowClient()
model_versions = client.search_model_versions(f"name='{MODEL_NAME}'")

print(f'Model: {MODEL_NAME}')
print(f'Versions registered: {len(model_versions)}')

for mv in model_versions:
    print(f"\nVersion: {mv.version}")
    print(f"Stage: {mv.current_stage}")
    print(f"Status: {mv.status}")